In [ ]:
import numpy as np
import random as r
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend

In [ ]:
def plot(I, O, **kwargs):
    count = 2 + len(kwargs)
    fig, axes = plt.subplots(count, 1, sharex=True,figsize=(12,3))
    ax1=axes[0]
    ax2=axes[1]

    ax1.step(time, I, 'b-') # y label
    ax2.step(time, O, 'r-')
   

    axes[-1].set_xlabel('time') # x label

    ax1.set_ylabel('Input', color='b')
    ax2.set_ylabel('Output', color='r')
    
    c = 2
    for k, v in kwargs.items():
        ax = axes[c]
        c+=1
        ax.step(time,v)
        ax.set_ylabel(k)
        
    plt.show()   

In [ ]:
# const
THRESHOLD_MIN = 1
THRESHOLD_MAX = 8
RECOVERY = 16
DROP = -2

class Neuron:
    def __init__(self, uid, tmax, recovery):
        self.uid = uid
        
        self.potential = 0
        self.threshold = THRESHOLD_MIN
        self.quiet_counter = 0
        self.recovery = recovery
        self.tmax=tmax

        self.targets = []
        self.inbox = 0
        
        self.fired = 0
    
    def enqueue(self, signal):
        self.inbox += signal
        
    def fire(self):
        self.fired = 1
        for t in self.targets:
            t.enqueue()

    def process(self, epoch):
        signal = self.inbox
        self.inbox = 0
        self.fired = 0
        
        def log(msg):
            if self.uid == 0:
                print(msg)
                
        if signal>0 and self.potential >=0:
            self.potential += signal
        elif self.potential > 0:
            self.potential -= 1
        elif self.potential < 0:
            self.potential +=1


        if self.potential >= self.threshold:
            
            self.potential = DROP
            self.threshold = min(self.threshold+1, self.tmax)
            
            self.quiet_counter = 0
            self.fire()
            

        if self.threshold > THRESHOLD_MIN:
            if self.quiet_counter >= self.recovery:
                self.threshold -=1
            else:
                self.quiet_counter += 1
 
                
class Cleft:
    def __init__(self, neuron, delay, signal):
        self.neuron = neuron
        self.delay = delay
        self.signal = signal
        
        
        self.inbox = [0] * delay
        self.pointer = 0
        
    def enqueue(self):
        self.inbox[self.pointer] = self.signal
        
    def process(self, epoch):
        self.pointer = (self.pointer + 1) % len(self.inbox)
        self.neuron.enqueue(self.inbox[self.pointer])
        self.inbox[self.pointer] = 0

In [ ]:
time=np.arange(0, 5000,1)
size = ((len(time)))
I=np.zeros(size)
O=np.zeros(size)
d1=np.zeros(size)
d2=np.zeros(size)

neurons = [Neuron(i, 20, 16) for i in range(500)]
# connect neurons randomly


fires = np.zeros((len(time), len(neurons)))
potential = np.zeros((len(time), len(neurons)))

clefts = []
for n in neurons:
    targets = r.sample(neurons, r.randint(1,12))
    for t in targets:
        c = Cleft(t, r.choice([1,1,2,2,3,4,5,6,7]), r.choice([-1,1,2,2,-2,3]))
        n.targets.append(c) 
        clefts.append(c)


for n in neurons[0:10]:
    n.enqueue(1)
for i in time:
    neuron_inbox = 0
    cleft_inbox = 0
    for n in neurons:
        pot = 30 if n.fired else n.potential
        
        potential[i][n.uid]=pot
        O[i]+=pot
        
        n.process(i)

    for c in clefts:
        c.process(i)

plt.figure(figsize = (20,5))
_ = plt.imshow(potential.T, interpolation='none', aspect='auto')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,5))
plt.autoscale(enable=True, axis='x', tight=True)
_ = plt.plot(O, lw=0.9)

In [ ]:
import subprocess
result = subprocess.run(['go', 'run', 'main.go'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(result.stdout.decode('utf-8'))